In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/train"
test_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/test"
val_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/val"

Mounted at /content/drive


In [3]:
#Gerador de imagens pré-processamento
image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

In [4]:
#Gerador dados treinamento e validação
train_generator = image_generator.flow_from_directory(
    train_dir,
    batch_size=8,
    shuffle=True,
    class_mode='binary',
    target_size=(180, 180)
)

validation_generator = image_generator.flow_from_directory(
    val_dir,
    batch_size=1,
    shuffle=False,
    class_mode='binary',
    target_size=(180, 180)
)


Found 5218 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
#Resnet
resnet_base_model = ResNet50(
    input_shape=(180, 180, 3),
    include_top=False,
    weights='imagenet'
)

94765736/94765736 [==============================] - 1s 0us/step


In [6]:
#Camadas personalizadas
resnet_model = tf.keras.Sequential([
    resnet_base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.6),
    Dense(128, activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

In [7]:
#Compilando Resnet
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
resnet_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=METRICS)

In [8]:
# Treinando modelo

# Calcular o peso das classes com base na contagem
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
num_pneumonia = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))

weight_for_0 = num_pneumonia / (num_normal + num_pneumonia)
weight_for_1 = num_normal / (num_normal + num_pneumonia)

class_weight = {0: weight_for_0, 1: weight_for_1}

history = resnet_model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    class_weight=class_weight,
    steps_per_epoch=100,
    validation_steps=25
)

Epoch 1/15
100/100 [==============================] - ETA: 0s - loss: 0.2812 - accuracy: 0.6175 - precision: 0.8541 - recall: 0.5980

100/100 [==============================] - 592s 6s/step - loss: 0.2812 - accuracy: 0.6175 - precision: 0.8541 - recall: 0.5980 - val_loss: 4105.1299 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
100/100 [==============================] - 525s 5s/step - loss: 0.2688 - accuracy: 0.6550 - precision: 0.8455 - recall: 0.6414
Epoch 3/15
100/100 [==============================] - 518s 5s/step - loss: 0.2711 - accuracy: 0.6363 - precision: 0.8588 - recall: 0.6221
Epoch 4/15
100/100 [==============================] - 517s 5s/step - loss: 0.2936 - accuracy: 0.5642 - precision: 0.7934 - recall: 0.5889
Epoch 5/15
100/100 [==============================] - 519s 5s/step - loss: 0.3253 - accuracy: 0.5365 - precision: 0.7242 - recall: 0.5687
Epoch 6/15
100/100 [==============================] - 527s 5s/step - loss: 0.2816 - accuracy: 0.5163 - precision: 0.7781 - recall: 0.5041
Epoch 7/15
100/100 [==============================] - 528s 5s/step - loss: 0.2517 - a

In [9]:
# Métricas para treinamento
train_metrics = resnet_model.evaluate(train_generator, steps=len(train_generator))
train_loss, train_accuracy, train_precision, train_recall = train_metrics

# Métricas para validação
val_metrics = resnet_model.evaluate(validation_generator, steps=len(validation_generator))
val_loss, val_accuracy, val_precision, val_recall = val_metrics

# Calcular F1 Score manualmente para treinamento
f1_train = 2 * (train_precision * train_recall) / (train_precision + train_recall)

# Calcular F1 Score manualmente para validação
f1_val = 2 * (val_precision * val_recall) / (val_precision + val_recall)

# Exibir métricas
print("Métricas de Treinamento:")
print(f"Loss: {train_loss:.4f}")
print(f"Acurácia: {train_accuracy * 100:.2f}%")
print(f"Precisão: {train_precision * 100:.2f}%")
print(f"Recall: {train_recall * 100:.2f}%")
print(f"F1 Score (Treinamento): {f1_train * 100:.2f}%")

print("\nMétricas de Validação:")
print(f"Loss: {val_loss:.4f}")
print(f"Acurácia: {val_accuracy * 100:.2f}%")
print(f"Precisão: {val_precision * 100:.2f}%")
print(f"Recall: {val_recall * 100:.2f}%")
print(f"F1 Score (Validação): {f1_val * 100:.2f}%")

16/16 [==============================] - 3s 190ms/step - loss: 1.4542 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.6250
Métricas de Treinamento:
Loss: 0.4361
Acurácia: 79.88%
Precisão: 92.59%
Recall: 79.26%
F1 Score (Treinamento): 85.41%

Métricas de Validação:
Loss: 1.4542
Acurácia: 50.00%
Precisão: 50.00%
Recall: 62.50%
F1 Score (Validação): 55.56%
